### **Part 1: Design the dimensional model**

Create a **dbt-style data model** to support analytics

Consider: How would someone use these tables to answer business questions about revenue, customer segments, and Organizations adoption?


## RIOT Analytics — dbt Project Structure


### A- Model Conception

For the dbt modeling organization, I'm following a hybrid approach that combines medallion architecture with star schema design.

Bronze and Silver layers follow the medallion pattern to ensure a structured flow of cleaning and validation transformations—raw data is ingested as-is (Bronze), then cleaned, typed, and validated (Silver).

Gold layer implements a star schema as the foundation for both ad-hoc analytical queries and upstream transformations. The star schema provides a clean, intuitive model for SQL analysts to explore data directly, while also serving as the base for more complex pre-calculated queries and business-wide tables optimized for BI tools.

Each layer is isolated in a separate schema (dataset) within the same BigQuery project, enabling:

   - Clear logical boundaries between raw, cleaned, and analytical data
   - Granular access control (analysts can query Gold without accessing Bronze)
   - Independent materialization strategies per layer
   - Cost tracking and performance monitoring by layer

![Data Flow Diagram](./annexe/dbt_model.png)


### Fact and dimension tables relationships

![Data Flow Diagram](./annexe/stripe_star_schema.png)

## B - Incremental model organization

In a real data use case. We won't have static raw data, but usually a x-periodic new or updated data. Dbt offer some hands on incremental strategies to avoid re-transforming each time the whole dbt-project. One critea for pipelines is to ensure idempotency while doing incremental process

### Idempotency Strategy

### Timestamp-Based Interval Processing

To ensure **idempotent and reproducible** data pipeline runs, all incremental models use **explicit date ranges** defined by `start_date` and `end_date` parameters rather than relying on high-water marks.

---

### Core Principle

**Every dbt run processes a specific, bounded time interval:**
- `start_date`: Beginning of the processing window (inclusive)
- `end_date`: End of the processing window (non-inclusive)
- Records selected: `WHERE date >= start_date AND date < end_date`

---

### Benefits

1. **Idempotency:** Running the same date range multiple times produces identical results
2. **Reproducibility:** Can reprocess any historical period on demand
3. **Testability:** Easy to test specific date ranges in development
4. **Debugging:** Clear boundaries for investigating data issues
5. **Backfill control:** Explicit control over which periods to process

### Incremental strategy

By default we will apply merge strategy where we define a primary key and we ensure insert only new data or updated one. In case of state change and when tracking those changes are important (maybe like fct_subscriptions) we might use scd2 incremental logic that will gave us a sort validity interval for each data.


## C - Storage Strategy

Medallion architecture consume a lot of storage space as data volume grows, we should take that into consideration for mid/long term. 

### Current Approach: BigQuery Native Tables

For the initial implementation, all layers are stored as **BigQuery native tables/views**:

| Layer | Storage Type | Rationale |
|-------|-------------|-----------|
| **Bronze** | BigQuery views | Zero storage cost, references seed data or source tables |
| **Silver** | BigQuery tables | Fast query performance, native BigQuery optimizations |
| **Gold** | BigQuery tables | Optimized for BI tools, leverages BigQuery caching |

**Benefit:**
- Simplicity of implementation since there is only one engine and less engineering between different ecosystems.


### Futur Approach: To take into account  ---> Apache Iceberg for Bronze/Silver

As data volume grows or requirements evolve, **Apache Iceberg** tables stored in **Cloud Storage** may be considered for Bronze and Silver layers:

**When to consider?**
- Multi-engine access patterns (Spark, Trino, Flink alongside BigQuery)
- Desire for cloud-agnostic, open table format to avoid vendor lock-in
- Data volume grows and storage costs become significant

**Hybrid architecture with Iceberg:**

**Gold layer remains in BigQuery native tables** for optimal BI performance.

**Bronze and silver moves to apache iceberg**

## D- Retention strategy

### Retention Policies by Layer

Different layers have different retention requirements based on their purpose and access patterns:

| Layer | Retention Period | Rationale | 
|-------|-----------------|-----------|
| **Bronze** | 90 days | Raw data replicas for reprocessing and auditing. Source of truth remains in operational systems (Stripe). |
| **Silver** | 2 years | Cleaned data for historical analysis and model retraining. Balances analytical needs with storage costs. |
| **Gold** | 5+ years | Business-critical aggregates and metrics for long-term trend analysis, compliance, and reporting. |


**Key principle:** Data becomes more valuable as it moves through the layers. Gold represents the refined, business-ready output that justifies extended retention for strategic analysis and regulatory compliance.

## E- Macros Strategy

### Purpose and Vision

Establish a **reusable macros library** organized by function to standardize transformations, ensure data quality and ease of peer programming. The mid term idea is to be able to pick existing macros if enough to reach our transformations goal or add additionnal ones if some missing logic transformations. The long-term goal is to create a sort of low code library powered by an agentic AI to reach a high % of automation, allowing analysts to generate complex analytical queries without writing SQL.

### Macro Categories

#### 1. **SQL Core Macros**
- **Purpose:** Fundamental building blocks for common SQL operations
- **Examples:** CTE generation, sql mandatory/classic function creation

#### 2. **Normalization Macros**
- **Purpose:** Standardize data formats and business logic
- **Examples:** A naming conventions (important in a long term view), date standardization ...


#### 3. **Quality & Compliance Macros**
- **Purpose:** Enforce data quality checks and regulatory requirements
- **Examples:** PII masking/hashing, GDPR deletion helpers, Filling algorithm, ..


#### 4. **Statistics Macros**
- **Purpose:** Generate common analytical calculations
- **Examples:** MRR calculations, churn rate formulas, cohort analysis, growth metrics or more technical sql calculation like window functions 




## F- Test Categories

## Testing Strategy

### Purpose and Vision

Implement a **comprehensive testing framework** using dbt's generic tests, organized by test type to ensure data quality, integrity, and business logic validation across all layers. The strategy prioritizes maintainability through clear folder organization and reusable test definitions.

---

DBT is intersting tool to do testing but the way of implementing it isn't optimal specially the fact when we want to store failures it automatically create a table by test which that the number of test tables will grow by trying to put more tests. The proposed idea is the make create a central table that will store necessary meta-data that concern the model and tested column + the result of the test that we want store. That way we can have one centralized table of tests (by layer) that will allow us to do quality data dashboards + easily database mangement. And ofc if needed we can create some table per test if need more information about the issue.

### Test Organization Structure

Tests are organized in a dedicated folder hierarchy by test type for easy discovery and maintenance:



#### 1. **Business Logic Tests**
- **Purpose:** Enforce domain-specific rules and constraints
- **Location:** `tests/generic/business_logic/`
- **Test types:** Custom singular tests for business rules
- **Applied to:** Silver and Gold layers where business logic is applied
- **Example validations:**
  - Active subscriptions must have MRR > 0
  - Canceled subscriptions must have MRR = 0
  - Customer creation date cannot be in the future
  - Multi-module customers must have 2+ active subscriptions

#### 2. **Data Quality Tests**
- **Purpose:** Validate data formats, ranges, and accepted values
- **Location:** `tests/generic/data_quality/`
- **Test types:** `accepted_values`, range checks, format validation, null checks
- **Applied to:** All layers, with strictest checks in Silver
- **Example validations:**
  - Subscription status is only 'active' or 'canceled'
  - Module names match approved list (Awareness, Simulation, Sonar, Slash)
  - MRR is non-negative
  - Email addresses follow valid format
  - Dates are within reasonable ranges

#### 3. **Reconciliation Tests**
- **Purpose:** Ensure data consistency across layers and aggregations
- **Location:** `tests/generic/reconciliation/`
- **Test types:** Custom cross-layer validation tests
- **Applied to:** Between Bronze→Silver, Silver→Gold transitions
- **Example validations:**
  - Row counts match between Bronze and Silver after cleaning
  - Total MRR in Gold equals sum of Silver subscription MRR
  - Customer counts reconcile across segmentation models
  - No data loss during transformations

---

### Test Execution by Layer

| Layer | Primary Focus | Key Test Types |
|-------|--------------|----------------|
| **Bronze** | Identity & completeness | Primary keys, not null, freshness |
| **Silver** | Data validity & relationships | All test types, strictest validation rules |
| **Gold** | Aggregation accuracy & grain | Reconciliation, business metrics validation, grain checks |

---

## G -Folder Organization

```
riot_analytics/
│
├── dbt_project.yml
├── packages.yml
│
│── seeds/
│      ├── sources.yml
│      ├── stripe_subscriptions.sql
│      └── stripe_customers.sql
│
│ ── macros
│       ├── sql_core
│       │   
│       ├── normalization
│       │ 
│       │── quality_compliance
│       │
│       └── stats
│
├── models/
│   │── schema.yml (tests + documentation) 
│   ├── silver/
│   │   ├── slv_stripe_subscriptions.sql
│   │   └── slv_stripe_customers.sql
│   │
│   └── gold/
│       │
│       ├── star/
│       │   ├── fct_subscriptions.sql 
│       │   ├── dim_customers.sql
│       │   └── dim_plans.sql
│       │
│       ├── wide/
|       │    ├── sales
|       │    │      
|       │    ├── product
|       │    │ 
|       │    │── cross
│       │   
│       │   
│       │
│       └── joins/
│           └── gld_subscriptions_customers.sql
│
│
└── tests/
    ├── assert_test3
    ├── assert_test2
    └── assert_test1
```

-------------------------------------------------------------------


### Part 2: Write SQL queries

Using the raw CSVs as source tables, write SQL to answer these business questions:


In [5]:
!pip install pandas

  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/2e/7c/870c7e7daec2a6c7ff2ac9e33b23317230d4e4e954b35112759ea4a924a7/pandas-3.0.1-cp311-cp311-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 3.2 MB/s eta 0:00:00
  Obtaining dependency information for numpy>=1.26.0 from https://files.pythonhosted.org/packages/1b/46/6fa4ea94f1ddf969b2ee941290cca6f1bfac92b53c76ae5f44afe17ceb69/numpy-2.4.2-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 45.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 43.7 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.2.1 -> 26.0.1
[notice] To update, run: pip install --upgrade pip


In [33]:
import sqlite3
import pandas as pd

# Connect to the database
conn = sqlite3.connect('./data/riot_analytics.db')


**Query A - Customer Segmentation:**

```markdown
Calculate total MRR by customer segment as of today:
- Organization customers (has organization_id)
- Standalone customers (no organization_id)
- Multi-module customers (2+ active subscriptions to different modules)

Show: segment_name, customer_count, total_mrr, avg_mrr_per_customer
```

In [34]:
query = """
WITH customer_subscriptions AS (
    SELECT 
        c.customer_id,
        c.metadata_organization_id,
        COUNT(DISTINCT s.module) AS distinct_module_count,
        SUM(CASE WHEN s.status = 'active' THEN s.mrr ELSE 0 END) AS total_mrr
    FROM stripe_customers c
    LEFT JOIN stripe_subscriptions s 
        ON c.customer_id = s.customer_id
        AND s.status = 'active'
    GROUP BY c.customer_id, c.metadata_organization_id
),
segmented AS (
    SELECT customer_id, total_mrr, 'Organization customers' AS segment_name
    FROM customer_subscriptions
    WHERE metadata_organization_id IS NOT NULL
    
    UNION ALL
    
    SELECT customer_id, total_mrr, 'Standalone customers' AS segment_name
    FROM customer_subscriptions
    WHERE metadata_organization_id IS NULL 
    
    UNION ALL
    
    SELECT customer_id, total_mrr, 'Multi-module customers' AS segment_name
    FROM customer_subscriptions
    WHERE distinct_module_count >= 2
)
SELECT
    segment_name,
    COUNT(DISTINCT customer_id) AS customer_count,
    SUM(total_mrr) AS total_mrr,
    ROUND(AVG(total_mrr), 2) AS avg_mrr_per_customer
FROM segmented
GROUP BY segment_name
ORDER BY total_mrr DESC;
"""

df = pd.read_sql_query(query, conn)

In [35]:
df

,segment_name,customer_count,total_mrr,avg_mrr_per_customer
0,Multi-module customers,15,41900,2793.33
1,Organization customers,15,38850,2590.00
2,Standalone customers,15,8400,560.00



**Query B - Module Performance:**

```markdown
For each module, calculate:
- Number of active subscriptions
- Total MRR from active subscriptions
- Average MRR per subscription
- Churn rate (% of all-time subscriptions that have been canceled)

Order by total MRR descending
```


In [ ]:
query = """
WITH module_metrics AS (
    SELECT
        module,
        COUNT(CASE WHEN status = 'active' THEN 1 END) AS active_subscriptions,  -- Active subscriptions
        SUM(CASE WHEN status = 'active' THEN mrr ELSE 0 END) AS total_mrr,
        ROUND(
            AVG(CASE WHEN status = 'active' THEN mrr END), 
            2
        ) AS avg_mrr_per_subscription, -- Average MRR per active subscription
        -- Churn rate
        COUNT(CASE WHEN status = 'canceled' THEN 1 END) AS canceled_subscriptions,
        COUNT(*) AS all_time_subscriptions,
        ROUND(
            COUNT(CASE WHEN status = 'canceled' THEN 1 END) * 100.0 / COUNT(*),
            2
        ) AS churn_rate_pct
    FROM stripe_subscriptions
    GROUP BY module
)
SELECT
    module,
    active_subscriptions,
    total_mrr,
    avg_mrr_per_subscription,
    churn_rate_pct
FROM module_metrics
ORDER BY total_mrr DESC;
"""

# Execute and fetch results
df = pd.read_sql_query(query, conn)
df

,module,active_subscriptions,total_mrr,avg_mrr_per_subscription,churn_rate_pct
0,Awareness,26,28150,1082.69,10.34
1,Sonar,6,7650,1275.00,0.00
2,Simulation,7,6400,914.29,12.50
3,Slash,6,5050,841.67,14.29



**Query C - Organization Analysis:**

```markdown
For each organization, show:
- Organization ID
- Number of workspaces (distinct customers in that org)
- Total MRR across all workspaces
- Comma-separated list of unique modules in use across the org
- Earliest workspace creation date (when did this org first join RIOT?)

Only include organizations with 2+ workspaces
```

In [ ]:
query = """
WITH org_data AS (
    SELECT
        c.metadata_organization_id AS organization_id,
        c.customer_id,
        c.created_at,
        s.module,
        s.mrr
    FROM stripe_customers c
    LEFT JOIN stripe_subscriptions s
        ON c.customer_id = s.customer_id
        AND s.status = 'active'
    WHERE c.metadata_organization_id IS NOT NULL 
),
org_metrics AS (
    SELECT
        organization_id,
        COUNT(DISTINCT customer_id) AS workspace_count,
        SUM(COALESCE(mrr, 0)) AS total_mrr,
        GROUP_CONCAT(DISTINCT module) AS modules_in_use,
        MIN(created_at) AS earliest_workspace_created
    FROM org_data
    GROUP BY organization_id
    HAVING COUNT(DISTINCT customer_id) >= 2
)
SELECT
    organization_id,
    workspace_count,
    total_mrr,
    modules_in_use,
    earliest_workspace_created
FROM org_metrics
ORDER BY total_mrr DESC;
"""

# Execute and fetch results
df = pd.read_sql_query(query, conn)
df

,organization_id,workspace_count,total_mrr,modules_in_use,earliest_workspace_created
0,org_052,2,9750,"Simulation,Awareness,Slash",2024-01-30
1,org_056,2,8300,"Simulation,Sonar,Awareness,Slash",2024-01-28
2,org_051,2,6350,"Sonar,Awareness,Slash",2024-01-25
3,org_055,2,5450,"Sonar,Awareness,Simulation",2024-04-15
4,org_053,2,3200,"Sonar,Awareness,Simulation",2024-01-10
5,org_050,3,2950,"Simulation,Awareness,Slash",2024-01-15
6,org_054,2,2850,"Awareness,Simulation",2024-04-20



### Part 3: Data Quality

What **dbt tests** would you add to ensure data quality?

List at least **6-8 tests** across your models. For each test, specify:

- Model/table name
- Column(s) being tested
- Test type (unique, not_null, relationships, accepted_values, custom, etc.)
- Why this test matters



1. Unique customer IDs (bronze layer)

- Model name: stripe_customers
- Column: customer_id
- Test type: unique, not_null
- Why this test matters: Customer ID is the primary key. We should ensure that there is no duplicates that will break downstream joins.


2. Unique subscription IDs (Bronze layer)

- Model name: stripe_subscriptions
- Column: subscription_id
- Test type: unique, not_null
- Why this test matters:Subscription ID is alse the primary key, duplicates might inflate MRR calculations and subscription counts.

3. Referential integrity: subscriptions and customers (Silver layer)

- Model name: slv_stripe_subscriptions
- Column: customer_id
- Test type: relationships
- Why this test matters: Every subscription must belong to a valid customer. Orphaned subscriptions for example might cause incorrect segmentation (since we will not find a corresponding organisation )

4. Non-negative MRR (Silver layer)

- model: slv_stripe_subscriptions
- column: mrr
- Test type: assert_non_negative
- Why it matters: Negative MRR is a data error that would understate revenue. Active subscriptions should have MRR ≥ 0 (canceled subs have MRR = 0).

5. Consistent module names
- Model name: slv_stripe_subscriptions
- Column: module
- Test type: accepted_values
- Why this test matters: Typos or case inconsistencies ('awareness', 'AWARENESS') would break multi-module customer identification and module-level reporting.

6. Row count reconcilation
- Model: fct_subscriptions
- Test type: reconcilation test (verify if fact subscriptions number and silver layer subscriptions layer are the same)
- Why it matters: If grain is subscription, row counts should match. Differences indicate filtering errors or duplicates.

7. Customer segmentation: All customers classified
- Model: gld_customer_segment (if created following dbt modelisation logic above)
- column: segment_type
- test type: not_null, accepted_values
- Why it matters: Every customer must be assigned to exactly one segment. NULLs or invalid values break segmentation reporting.